In [1]:
import pandas as pd
import re
import cleaning_functions as cf

In [2]:
oscars = pd.read_csv('../data/Clean/oscars_cleaned.csv')

In [4]:
oscars.head(2)

,year_film,year_ceremony,Oscars nominations,name,film,Oscars winner
0,1967,1968,Actor in a leading role,Warren Beatty,Bonnie and Clyde,False
1,1967,1968,Actor in a leading role,Dustin Hoffman,The Graduate,False


In [ ]:
golden = pd.read_csv('../data/Clean/oscars_cleaned.csv')